In [ ]:
%env
SCHEMA=https://
OUTPUT=s3://
ETL_CONF_ENV=production

In [ ]:
{
  "type": "DelimitedExtract",
  "name": "extract data from green_tripdata schema 0",
  "environments": ["production", "test"],
  "inputURI": "s3://nyc-tlc/trip data/green_tripdata_2013-08.csv",
  "outputView": "green_tripdata0_raw",            
  "delimiter": "Comma",
  "quote" : "DoubleQuote",
  "header": true,
  "persist": true
}

In [ ]:
{
  "type": "TypingTransform",
  "name": "apply green_tripdata schema 0 data types",
  "environments": ["production", "test"],
  "schemaURI": ${SCHEMA},
  "inputView": "green_tripdata0_raw",            
  "outputView": "green_tripdata0"
}

In [ ]:
{
  "type": "ParquetLoad",
  "name": "write out green_tripdata0 dataset",
  "environments": ["production", "test"],
  "inputView": "green_tripdata0",
  "outputURI": ${OUTPUT},
  "saveMode": "Overwrite"
}